<a href="https://colab.research.google.com/github/shradha14-94/Letsupgrade-Data-science-B9/blob/main/capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name - Shradha Gilbile

In [ ]:
#CAPSTONE PROJECT: COVID19 CT SCAN ANALYSIS

This example will show the steps needed to build a 3D convolutional neural network (CNN) to predict the presence of viral pneumonia in computer tomography (CT) scans. 2D CNNs are commonly used to process RGB images (3 channels). A 3D CNN is simply the 3D equivalent: it takes as input a 3D volume or a sequence of 2D frames (e.g. slices in a CT scan), 3D CNNs are a powerful model for learning representations for volumetric data.



In [ ]:
import glob
import pandas as pd
import os

This dataset consists of lung CT scans with COVID-19 related findings, as well as without such findings.

We will be using the associated radiological findings of the CT scans as labels to build a classifier to predict presence of viral pneumonia. Hence, the task is a binary classification problem.

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading data and preprocessing CT scans store raw voxel intensity in Hounsfield units (HU). They range from -1024 to above 2000 in this dataset. Above 400 are bones with different radiointensity, so this is used as a higher bound. A threshold between -1000 and 400 is commonly used to normalize CT scans.

To process the data, we do the following:

We first rotate the volumes by 90 degrees, so the orientation is fixed We scale the HU values to be between 0 and 1. We resize width, height and depth. Here we define several helper functions to process the data. These functions will be used when building training and validation datasets

In [ ]:
path=[]
label=[]
for i in glob.glob('/content/drive/MyDrive/capstone_project/'+'*/*.png'):
  path.append(i)
  label.append(i.split('/')[-2])

In [ ]:
path

['/content/drive/MyDrive/capstone_project/covid/Covid (25).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (227).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (20).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (130).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (223).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (129).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (185).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (256).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (187).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (138).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (203).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (160).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (1266).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (146).png',
 '/content/drive/MyDrive/capstone_project/covid/Covid (214).png',
 '/content/

In [ ]:
data = pd.DataFrame({"Path":path,"Label":label})
data

,Path,Label
0,/content/drive/MyDrive/capstone_project/covid/...,covid
1,/content/drive/MyDrive/capstone_project/covid/...,covid
2,/content/drive/MyDrive/capstone_project/covid/...,covid
3,/content/drive/MyDrive/capstone_project/covid/...,covid
4,/content/drive/MyDrive/capstone_project/covid/...,covid
...,...,...
2497,/content/drive/MyDrive/capstone_project/noncov...,noncovid
2498,/content/drive/MyDrive/capstone_project/noncov...,noncovid
2499,/content/drive/MyDrive/capstone_project/noncov...,noncovid
2500,/content/drive/MyDrive/capstone_project/noncov...,noncovid


In [ ]:
data['Label'].value_counts()

covid       1273
noncovid    1229
Name: Label, dtype: int64

In [ ]:
master_data=data.sample(frac=1)

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Dense,Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

Data augmentation The CT scans also augmented by rotating at random angles during training. Since the data is stored in rank-3 tensors of shape (samples, height, width, depth), we add a dimension of size 1 at axis 4 to be able to perform 3D convolutions on the data. The new shape is thus (samples, height, width, depth, 1). There are different kinds of preprocessing and augmentation techniques out there, this example shows a few simple ones to get started.

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

In [ ]:
test_generator = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
master_data['Label'] = master_data['Label'].replace({"COVID":0,"non-COVID":1})

In [ ]:
master_data['Label'].unique()

array(['noncovid', 'covid'], dtype=object)

In [ ]:
master_data.head(5)

,Path,Label
2084,/content/drive/MyDrive/capstone_project/noncov...,noncovid
719,/content/drive/MyDrive/capstone_project/covid/...,covid
265,/content/drive/MyDrive/capstone_project/covid/...,covid
1845,/content/drive/MyDrive/capstone_project/noncov...,noncovid
2150,/content/drive/MyDrive/capstone_project/noncov...,noncovid


In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=master_data,
    x_col='Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=4,
    shuffle=True,
    subset='training'
)

Found 2002 validated image filenames.


In [ ]:
val_images = train_generator.flow_from_dataframe(
    dataframe=master_data,
    x_col='Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=4,
    shuffle=True,
    subset='validation'
)

Found 500 validated image filenames.


In [ ]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [ ]:
# Import The Libraries 

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential


import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
path= []
label = []
for i in glob('/content/drive/MyDrive/capstone_project'+'*/*.png'):
    path.append(i)
    label.append(i.split('/')[-2])

In [ ]:
import pandas as pd

In [ ]:
# Path
capstone_project=("/content/drive/MyDrive/capstone_project")
covid_path=("/content/drive/MyDrive/capstone_project/covid")
noncovid_path=("/content/drive/MyDrive/capstone_project/noncovid")

In [ ]:
# Set Resize variable
IMAGE_SIZE = [224, 224] # This is my desired image size... and also ResNet50 accepts image of 224*224.

In [ ]:
resnet = ResNet50(
    input_shape = IMAGE_SIZE + [3], # Making the image into 3 Channel, so concating 3.
    weights = 'imagenet', # Default weights.
    include_top = False   # 
)

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
for layer in resnet.layers:
    layer.trainable = True

In [ ]:
folders = glob("/content/drive/MyDrive/capstone_project" + '/*')
folders

['/content/drive/MyDrive/capstone_project/covid',
 '/content/drive/MyDrive/capstone_project/noncovid']

In [ ]:
len(folders)

2

In [ ]:
capstone_project_label = ['covid','noncovid']

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
len(folders)

2

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
len(folders)

2

In [ ]:
prediction = Dense(len(folders), activation = 'softmax')(x)

In [ ]:
resnet.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>

In [ ]:
# Create a model Object
model = Model(inputs = resnet.input, outputs = prediction)

Here the model accuracy and loss for the training and the validation sets are plotted. Since the validation set is class-balanced, accuracy provides an unbiased representation of the model's performance.

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.compile (
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
nb_classes = 1010
batch_size = 64
img_size = 200
nb_epochs = 30

In [ ]:

train_datagen=ImageDataGenerator(rescale=1./255, 
    validation_split=0.25,
    horizontal_flip = True,    
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3
    )

train_generator=train_datagen.flow_from_dataframe(
    dataframe=master_data,
    directory="/content/drive/MyDrive/capstone_project",
    x_col="Path",
    y_col= "Label",
    batch_size=4,
    shuffle=True,
    class_mode="categorical", 
    target_size=(img_size,img_size) 
    )

Found 2502 validated image filenames belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=master_data,
    directory="/content/drive/MyDrive/capstone_project",
    x_col="Path",
    y_col="Label",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",    
    target_size=(img_size,img_size))

Found 2502 validated image filenames belonging to 2 classes.


In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
ckpt_path = 'new_model.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(ckpt_path,save_best_only=True)

In [ ]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(patience=4)

In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data = test_generator,
    epochs = 5,
    steps_per_epoch = len(train_generator),
    validation_steps = len(test_generator)
)      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
626/626 [==============================] - 1965s 3s/step - loss: 2.5528 - accuracy: 0.5663 - val_loss: 1.0650 - val_accuracy: 0.5280
Epoch 2/5
626/626 [==============================] - 61s 97ms/step - loss: 0.6897 - accuracy: 0.6787 - val_loss: 0.7620 - val_accuracy: 0.6091
Epoch 3/5
626/626 [==============================] - 62s 99ms/step - loss: 0.6083 - accuracy: 0.7302 - val_loss: 0.6292 - val_accuracy: 0.8122
Epoch 4/5
626/626 [==============================] - 60s 96ms/step - loss: 0.5674 - accuracy: 0.7526 - val_loss: 3.1189 - val_accuracy: 0.5120
Epoch 5/5
626/626 [==============================] - 60s 96ms/step - loss: 0.5059 - accuracy: 0.7758 - val_loss: 1.0967 - val_accuracy: 0.6111


In [ ]:
prediction = model.predict(test_generator)

In [ ]:
prediction

array([[0.9987734 , 0.00122654],
       [0.5295515 , 0.47044852],
       [0.12521604, 0.874784  ],
       ...,
       [0.96640146, 0.03359851],
       [0.96505576, 0.03494432],
       [0.9645679 , 0.0354321 ]], dtype=float32)

In [ ]:
np.argmax(prediction, axis = 1)

In [ ]:
prediction = np.argmax(prediction, axis = 1)
prediction